In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "your-api-key"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [ ]:
prompt.format(schema="my schema", question="How many artists are there?")

In [ ]:
from langchain_community.utilities import SQLDatabase

# if you are using MySQL
mysql_uri = 'mysql+mysqlconnector://root:<your-mysql-password>@localhost:3306/<database-name>'

db = SQLDatabase.from_uri(mysql_uri) 

In [ ]:
def get_schema(_):
    schema = db.get_table_info()
    return schema

In [ ]:
get_schema(None)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

# stroutputparser to pass our string to the sql query, the next step in our llm chain

In [ ]:
try:
    sql_query = sql_chain.invoke({"question": "How many artists are there?"})
    print("Generated SQL Query:", sql_query)
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [ ]:
def run_query(query):
    return db.run(query)

In [ ]:
#Example from the database, change it freely
run_query("SELECT COUNT(*) AS TotalAlbums\nFROM Album;")

In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
    |StrOutputParser()
)

In [ ]:
try:
  full_chain.invoke({"question": "how many artists are there?"})
except Exception as e:
  print(f"An error occurred: {e}")